In [ ]:
import numpy as np
from sklearn.datasets import make_classification


In [ ]:
class Contextual_Bandit_generator:

    #@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    def __init__(self, config):
        self.state = {'X':[],'article':[],'reward':[]}
        self.population = [[],[]]
        self.population[0], self.population[1] = make_classification(
            n_samples=config['n_samples'],
            n_features=5,
            n_informative=5,
            n_classes=5,
            n_redundant=0,
            n_repeated=0,
            flip_y=0.2,
            shift = [10,4,0,0,0],
            scale = [7,1,1,1,1],
            random_state=8
        )
        self.population[0] = self.feature_engineer(self.population[0])
        self.current_context = {'X':[],'best_article':[]}
        
    #@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    def feature_engineer(self, Data):
        Data[:,0] = (Data[:,0]>0)*1
        Data[:,1] = (Data[:,1]>0)*1
        Data[:,3][(Data[:,3]<0)] = 0
        Data[:,3][(Data[:,3]>5)] = 5
        Data[:,3] = np.round(Data[:,3])
        Data[:,4] = np.round(Data[:,4])
        return Data

    #@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    def step(self):
        sample = np.random.choice(len(self.population[1]))
        x = self.population[0][sample]
        ba = self.population[1][sample]
        if len(self.state['X']) == 0:
            self.state['X'] = x
        else:
            self.state['X'] = np.vstack((self.state['X'],x))
        self.current_context['X'] = x
        self.current_context['best_article'] = ba

    #@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    def reward(self,a):        
        if self.current_context['best_article'] == a:
            response = np.random.binomial(n = 1, p = 0.9,size = 1)[0]
        else:
            response = np.random.binomial(n = 1, p = 0.05,size = 1)[0]
        if len(self.state['X']) == 0:      
            self.state['article'] = np.array(a)
            self.state['reward'] = np.array(response)
        else:
            self.state['article'] = np.append(self.state['article'],a)
            self.state['reward'] = np.append(self.state['reward'],response)
        

In [ ]:
#intended setup
env_config = {'n_samples':100000}
env = Contextual_Bandit_generator(env_config)


In [ ]:
#intended use
env.step()
a = 2
env.reward(a)
env.state
